#  Výdaje státního rozpočtu na kulturu

V tomto návodu se pomocí podíváme na výdaje státního rozpočtu z aplikace MONITOR pomocí odvětvového třídění a to sice v jazyce SQL. Proto musíme mít data nejdříve v databázi, kam je dostaneme například pomocí našeho NodeJS skriptu. Odvětvové třídění je dělení výdajů pomocí takzvaných paragrafů. Ty jsou hierarchicky uspořádané do pododdílů, oddílů a skupin. Jak vypadá třídění paragrafů vidíte v tabulce `c_paragraf` pomocí následujícího příkazu:

In [14]:
SELECT * FROM monitor.c_paragraf LIMIT 10;

(10 row(s) affected)

Total execution time: 00:00:00.139

paragraf,skupina,oddil,pododdil,paragraf_nazev,start_date,end_date
0000,0,0,0,Pro příjmy (technický záznam),1900-01-01,9999-12-31
1011,1,10,101,"Udržování výrobního potenciálu zemědělství, zemědělský půdní fond a mimoprodukční funkce zemědělství",1900-01-01,9999-12-31
1012,1,10,101,Podnikání a restrukturalizace v zemědělství a potravinářství,1900-01-01,9999-12-31
1013,1,10,101,"Genetický potenciál hospodářských zvířat, osiv a sádí",1900-01-01,9999-12-31
1014,1,10,101,"Ozdravování hospodářských zvířat, polních a speciálních plodin a zvláštní veterinární péče",1900-01-01,9999-12-31
1019,1,10,101,Ostatní zemědělská a potravinářská činnost a rozvoj,1900-01-01,9999-12-31
1021,1,10,102,Organizace trhu s produkty rostlinné výroby,1900-01-01,9999-12-31
1022,1,10,102,Organizace trhu s výrobky vzniklými zpracováním produktů rostlinné výroby,1900-01-01,9999-12-31
1023,1,10,102,Organizace trhu s produkty živočišné výroby,1900-01-01,9999-12-31
1024,1,10,102,Organizace trhu s výrobky vzniklými zpracováním produktů živočišné výroby,1900-01-01,9999-12-31


Nás může zajímat například pododdíl 331, který slučuje paragrafy týkající se výdajů na **kulturu**:

In [16]:
SELECT * FROM monitor.c_paragraf WHERE pododdil = '331';

(8 row(s) affected)

Total execution time: 00:00:00.113

paragraf,skupina,oddil,pododdil,paragraf_nazev,start_date,end_date
3311,3,33,331,Divadelní činnost,1900-01-01,9999-12-31
3312,3,33,331,Hudební činnost,1900-01-01,9999-12-31
3313,3,33,331,"Filmová tvorba, distribuce, kina a shromažďování audiovizuálních archiválií",1900-01-01,9999-12-31
3314,3,33,331,Činnosti knihovnické,1900-01-01,9999-12-31
3315,3,33,331,Činnosti muzeí a galerií,1900-01-01,9999-12-31
3316,3,33,331,Vydavatelská činnost,1900-01-01,9999-12-31
3317,3,33,331,Výstavní činnosti v kultuře,1900-01-01,9999-12-31
3319,3,33,331,Ostatní záležitosti kultury,1900-01-01,9999-12-31


Nás by ale teď zajímalo, kolik která instituce na kulturu dává. Na to máme data v tabulce `mis_ris`. Její strukturu si můžeme vypsat třeba pomocí schématu `information_schema` následovně:

In [2]:
SELECT 
   table_name, 
   column_name, 
   data_type   
FROM 
   information_schema.columns
WHERE 
   table_schema = 'monitor' AND table_name = 'mis_ris';

(18 row(s) affected)

Total execution time: 00:00:00.109

table_name,column_name,data_type
mis_ris,year,smallint
mis_ris,month,smallint
mis_ris,fiscper,character varying
mis_ris,zc_ucjed,character varying
mis_ris,zc_ico,character varying
mis_ris,fm_area,character varying
mis_ris,zfunds_ct,character varying
mis_ris,zcmmt_itm,character varying
mis_ris,func_area,character varying
mis_ris,zc_fund,character varying


Z dokumentace Monitoru zjistíme, co znamenají pole `zu_rozsch`, `zu_rozpzm` a `zu_rozkz` a následně si tabulku napojíme na předchozí číselník paragrafů. Dále si můžeme připojit číselník účetnách jednotek \`c\_ucjed\`, abychom věděli, které IČO patří kterí organizaci. Jak vidíme, nejvíce na kulturu dává očekávaně Ministerstvo kultury, pak Ministerstvo školství, ale vcelku překvapivě je na třetí příčce Ministerstvo zemědělství.

In [2]:
SELECT
    P.pododdil,
    zc_ico,
    MAX(U.ucjed_nazev) AS nazev,
    SUM(zu_rozsch) AS rozpocet_schvaleny,
    SUM(zu_rozpzm) AS rozpocet_upraveny,
    SUM(zu_rozkz) AS rozpocet_skutecnost
FROM remote_monitor.mis_ris AS D

-- připojení rozpočtových dat z tabulky mis_ris. zde pozor na pole start_date a end date, ktere urcuji platnost
LEFT JOIN remote_monitor.c_paragraf AS P ON SUBSTR(D.func_area,1,4) = P.paragraf AND MAKE_DATE(D.year,1,1) BETWEEN P.start_date AND P.end_date

-- připojení číselníku účetních jednotek
LEFT JOIN remote_monitor.c_ucjed AS U ON U.ico = D.zc_ico AND MAKE_DATE(D.year,1,1) BETWEEN U.start_date AND U.end_date

-- zde omezíme na pododdíl kultury a konec roku 2012
WHERE P.pododdil = '331' AND year = 2019 AND month = 12

-- sčítáme všechny položky v pododdílu, takže musíme říct, podle kterých údajů se má seskupovat. zde jsou to první dva sloupce
GROUP BY 1,2

(6 row(s) affected)

Total execution time: 00:00:07.682

pododdil,zc_ico,nazev,rozpocet_schvaleny,rozpocet_upraveny,rozpocet_skutecnost
331,00000000,NULL,0.00,0.00,0.00
331,00020478,Ministerstvo zemědělství,86240000.00,89337248.41,241414833.99
331,00022985,"Ministerstvo školství, mládeže a tělovýchovy",157905256.00,215025249.00,216014133.00
331,00023671,Ministerstvo kultury,7780973613.00,9879717090.62,8490386576.20
331,60162694,Ministerstvo obrany,0.00,19919740.00,19915040.00
331,60498030,Muzeum Policie České republiky,18924948.00,20358170.37,17175978.96


Jistě vás napadne, na co asi Ministerstvo zemědělství dává tolik peněz v odvětví kultury. Tak si zobrazíme větší detail, v rozpadu přímo na paragraf. A vidíme, že se jedná o muzea. Ministerstvo zemědělství totiž nejspíš touto částkou financuje zemědělská muzea, které jsou po celé ČR.

In [10]:
SELECT
    P.paragraf,
    MAX(P.paragraf_nazev) AS paragraf_nazev,
    SUM(zu_rozsch) AS rozpocet_schvaleny,
    SUM(zu_rozpzm) AS rozpocet_upraveny,
    SUM(zu_rozkz) AS rozpocet_skutecnost
FROM remote_monitor.mis_ris AS D
LEFT JOIN remote_monitor.c_paragraf AS P ON SUBSTR(D.func_area,1,4) = P.paragraf AND MAKE_DATE(D.year,1,1) BETWEEN P.start_date AND P.end_date
WHERE P.pododdil = '331' AND year = 2019 AND month = 12 AND zc_ico = '00020478'
GROUP BY 1

(2 row(s) affected)

Total execution time: 00:00:04.025

paragraf,paragraf_nazev,rozpocet_schvaleny,rozpocet_upraveny,rozpocet_skutecnost
3314,Činnosti knihovnické,0.00,353000.00,353000.00
3315,Činnosti muzeí a galerií,86240000.00,88984248.41,241061833.99
